# <strong style="color: tomato;">Image Processing</strong> $\color{blue}{\text{}}$
---
<!-- <style>.title {color: tomato;font-weight: bold;}.section {color: yellowgreen;}.subsection {color: royalblue;}</style> -->
<!-- # <span class="title">Image Processing</span> $\color{blue}{\text{}}$ -->

## <span style="color: yellowgreen;">1. </span>Introduction.

Goals of the section:
- learn various image processing operations,
- perform operations such as: Smoothing, Blurring, Morphological Operations,
- grab properties such as color spaces and histograms.

## <span style="color: yellowgreen;">2. </span>Color mappings.

Colorspaces:
- RGB - cube model,
- HLS (Hue, Lightnes, Saturation) - cylinder model with white as max lightnes no mater H / S,
- HSV (Hue, Saturation, Value) - cylinder model with white as a center of the cylinder no matter H / V.
We will not be using the HSL or HSV based color images

To change the colorspace we use the cvtColor() method.

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
img = cv2.imread('./Computer-Vision-with-Python/DATA/00-puppy.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
plt.imshow(img)

## <span style="color: yellowgreen;">3. </span>Blending and pasting images.

### <span style="color: royalblue;">a) </span>Blending part one

To blend images we use the addWeighted() function that uses both images and combines them. 

&ensp;&ensp;&ensp;&ensp;<strong>addWeighted() can only be used when the images are the same size!!</strong>

The formula used to blend: $\text{} new\_pixel = \alpha\times\text{pixel\_1} + \beta\times\text{pixel\_2} + \gamma$

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
img1 = cv2.imread('./Computer-Vision-with-Python/DATA/dog_backpack.png')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

img2 = cv2.imread('./Computer-Vision-with-Python/DATA/watermark_no_copy.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(img1)
# plt.imshow(img2)

Blending images of the same size:

In [ ]:
# resize the images to be the same size
img1 = cv2.resize(img1, (1200, 1200))
img2 = cv2.resize(img2, (1200, 1200))

In [ ]:
# src1, alpha, src2, beta, gamma
blended = cv2.addWeighted(img1, .8, img2, .2, 0)
plt.imshow(blended)

Blending images of different sizes:

- Overlay the small image on top of a larger image <strong style="font-size: 120%">(NO BLENDING)</strong>.
NumPy reassignment:

In [ ]:
img1 = cv2.imread('./Computer-Vision-with-Python/DATA/dog_backpack.png')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

img2 = cv2.imread('./Computer-Vision-with-Python/DATA/watermark_no_copy.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

In [ ]:
img2 = cv2.resize(img2, (600, 600))

large_img = img1
small_img = img2

In [ ]:
# defining the coordinates where we want to insert the small image
x_offset = 0
y_offset = 0

# where we will end slicing
x_end = x_offset + small_img.shape[1]
y_end = y_offset + small_img.shape[0]

In [ ]:
large_img[y_offset:y_end, x_offset:x_end] = small_img
plt.imshow(large_img)

### <span style="color: royalblue;">b) </span>Blending part two

ROI - Region Of Interest

- Blend together images of different sizes:

In [ ]:
img1 = cv2.imread('./Computer-Vision-with-Python/DATA/dog_backpack.png')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

img2 = cv2.imread('./Computer-Vision-with-Python/DATA/watermark_no_copy.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

img2 = cv2.resize(img2, (600, 600))

# creating ROI on larger image where we want to blend the smaller image
y_offset = img1.shape[0] - img2.shape[0]
x_offset = img1.shape[1] - img2.shape[1]

rows, cols, channels = img2.shape
roi = img1[y_offset:img1.shape[0], x_offset:img1.shape[1]]
plt.imshow(roi)

Creating the mask:

In [ ]:
img2gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
# plt.imshow(img2gray, 'gray')

mask_inv = cv2.bitwise_not(img2gray) # change 1 to 0
plt.imshow(mask_inv, 'gray')

Fixing the mask to include all the color channels, because now it is a 2D array

In [ ]:
import numpy as np
white_bgc = np.full(img2.shape, 255, np.uint8)

In [ ]:
bgc = cv2.bitwise_or(white_bgc, white_bgc, mask=mask_inv)
bgc.shape
plt.imshow(bgc)

In [ ]:
fg = cv2.bitwise_or(img2, img2, mask=mask_inv)
plt.imshow(fg)

In [ ]:
final_roi = cv2.bitwise_or(roi, fg)
plt.imshow(final_roi)

In [ ]:
large_img = img1
small_img = final_roi

large_img[y_offset:y_offset+small_img.shape[0], x_offset:x_offset+small_img.shape[1]] = small_img
plt.imshow(large_img)

## <span style="color: yellowgreen;">4. </span>Image thresholding.

It is a very simple method of segmenting images into different parts. Thresholding will convert an image to consist of only two valurs - B&W (binary)

Thresholding in OpenCV:

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# when passing the 0 as a second parameter we read the img in grayscale
img = cv2.imread('./Computer-Vision-with-Python/DATA/rainbow.jpg', 0)
plt.imshow(img, 'gray')

Different threshold types:
- cv2.THRESH_BINARY - change to 0/1 (above thresh => 1, below thresh => 0),
- cv2.THRESH_BINARY_INV - change to do 1/0,
- cv2.THRESH_TRUNC - above the threshold => threshold, below thresh => keep the original value,
- cv2.THRESH_TOZERO - above thresh => keep original, below => 0,
- cv2.THRESH_TOZERO_INV - above thresh => 0, below => keep original.

In [ ]:
# source image, threshold, max value expected, type of threshold
# any values above the threshold are set to 1, and any below are set to 0
# 127 = floor(255 / 2)
# maxval = 255 
ret, thresh1 = cv2.threshold(img,127, 255, cv2.THRESH_BINARY_INV)
plt.imshow(thresh1, 'gray')

In [ ]:
img = cv2.imread('./Computer-Vision-with-Python/DATA/crossword.jpg', 0)
plt.imshow(img, 'gray')

In [ ]:
def showPic(img):
    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111)
    plt.imshow(img, cmap='gray')

In [ ]:
showPic(img)

In [ ]:
# we have to play with either the threshold typy or a thresh valur
ret, th1 = cv2.threshold(img, 170, 255, cv2.THRESH_BINARY)
showPic(th1)

Links from the lecture to further read about the adaptive thresholding:
<div style="font-size: 3px">
Adaptive Threshold

https://stackoverflow.com/questions/28763419/adaptive-threshold-parameters-confusion

    @param src Source 8-bit single-channel image.
    .   @param dst Destination image of the same size and the same type as src.
    .   @param maxValue Non-zero value assigned to the pixels for which the condition is satisfied
    .   @param adaptiveMethod Adaptive thresholding algorithm to use, see #AdaptiveThresholdTypes.
    .   The #BORDER_REPLICATE | #BORDER_ISOLATED is used to process boundaries.
    .   @param thresholdType Thresholding type that must be either #THRESH_BINARY or #THRESH_BINARY_INV,
    .   see #ThresholdTypes.
    .   @param blockSize Size of a pixel neighborhood that is used to calculate a threshold value for the
    .   pixel: 3, 5, 7, and so on.
    .   @param C Constant subtracted from the mean or weighted mean (see the details below). Normally, it
    .   is positive but may be zero or negative as well.
</p>

In [ ]:
# BUT we can also automatically adjust the threshold type (adaptive thresholding)
# src; maxval; type of adaptive thresh; type of thresh to adapt; block size - pixel neigbourhood, has to be odd number; C constant
# th2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 5, 9)
th2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 3, 8)
showPic(th2)

In [ ]:
blended = cv2.addWeighted(th1, .6, th2, .4, 0)
showPic(blended)

## <span style="color: yellowgreen;">5. </span>Blurring and Smoothing.

General information:
- Smoothing can help get rid of noise or help an applicaton focus on general details.
- There are many methods of blurring and smoothing
- It is often combined with edge detection
- Edge detection can detect too many edges if the image has too high resolution without blurring

Methods we will be exploring:
- Gamma Correction
    - Gamma Correction can be applied to an image to make it appear brighter / darker depending on the Gamma value chosen
- Kernel based filters
    - kernels can be applied over an image to produce a variety of effects
    - the best way to explain this is through an [interactie visualization](http://setosa.io/ev/image-kernels/)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# convenient function 1
def loadImg():
    img = cv2.imread('./Computer-Vision-with-Python/DATA/bricks.jpg').astype(np.float32) / 255
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
# convenient function 2
def displayImg(img):
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111)
    ax.imshow(img)

In [ ]:
i = loadImg()
displayImg(i)

Gamma correction:

In [ ]:
gamma = 3 # gamma values over 1 => make the image darker
gamma = 1/4 # gamma values less than 1 => make the image brighter
result = np.power(i, gamma)
plt.imshow(result)

Blurring using Low Pass Filter:

In [ ]:
img = loadImg()
font = cv2.FONT_HERSHEY_COMPLEX
cv2.putText(img, 'bricks', (10, 600), font, 10, (255, 0, 0), 4)

displayImg(img)

In [ ]:
img = loadImg()
font = cv2.FONT_HERSHEY_COMPLEX
cv2.putText(img, 'bricks', (10, 600), font, 10, (255, 0, 0), 4)

kernel = np.ones((5, 5), dtype=np.float32) / 25
kernel

Applying a 2D filter

In [ ]:
# source image; desired depth => -1 means assign the input depth to the desired output (input depth = output depth); kernel
dst = cv2.filter2D(img, -1, kernel)
displayImg(dst)

In [ ]:
# RESET
img = loadImg()
font = cv2.FONT_HERSHEY_COMPLEX
cv2.putText(img, 'bricks', (10, 600), font, 10, (255, 0, 0), 4)
print('reset')

Default blur kernel:

In [ ]:
# src; kernel size;
blurred = cv2.blur(img, (5, 5))
displayImg(blurred)

Gausian blurring:

In [ ]:
# src; kernel size; sigma value => standard deviation
blurred_gauss = cv2.GaussianBlur(img, (5, 5), 10)
displayImg(blurred_gauss)

Median blurring (good for reducing image noise):

In [ ]:
# src; kernel size (must be square size); 
blurred_median = cv2.medianBlur(img, 5)
displayImg(blurred_median)

In [ ]:
img = cv2.imread('./Computer-Vision-with-Python/DATA/sammy.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
displayImg(img)

In [ ]:
noise_img = cv2.imread('./Computer-Vision-with-Python/DATA/sammy_noise.jpg')
# displayImg(noise_img)

corrected_noise = cv2.medianBlur(noise_img, 5)
displayImg(corrected_noise)

Bilateral filtering:

In [ ]:
# reduce unwanted noise and keep edges fairly sharp BUT it is usually slower than other filters
blur = cv2.bilateralFilter(img, 9, 75, 75)
displayImg(blur)

## <span style="color: yellowgreen;">6. </span>Morphological operators.

- These operators are sets of Kernerls that can achieve a variety of  efects like reducing noise.
- Certain operators are very good at reducing black points on a white background (and vice versa).
- Certain operators can also achieve an erosion and dilation effect that can add or erode from an existing image.
    - This effect can be seen best on text data so we will practice various operators on some simple white text on black background.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def loadImage():
    blank_img = np.zeros((600, 600))
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(blank_img, text='ABCDE', org=(50, 300), fontFace=font, fontScale=5, color=(255, 255, 255), thickness=25, lineType=cv2.LINE_AA)
    return blank_img

In [ ]:
def displayImage(img):
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap='gray')

In [ ]:
img = loadImage()
displayImage()

Erosion - erodes away the boundaries of foreground objects:

In [ ]:
kernel = np.ones((5, 5), dtype=np.uint8)
result = cv2.erode(img, kernel, iterations=4)
displayImage(result)

Opening -> erosion + dilation (usefull to remove background noise):

In [ ]:
img = loadImage()
white_noise = np.random.randint(0, 2, (600, 600)) * 255 # make the matrix of values 255 and 0
noise_img = img + white_noise
displayImage(noise_img)

In [ ]:
opening = cv2.morphologyEx(noise_img, cv2.MORPH_OPEN, kernel)
displayImage(opening)

Foreground noise:

In [ ]:
img = loadImage()
black_noise = np.random.randint(0, 2, (600, 600))
black_noise *= -255
black_noise_img = img + black_noise
black_noise_img[black_noise_img == -255] = 0
displayImage(black_noise_img)

Closing - good at getting rid of the foreground noise:

In [ ]:
closing = cv2.morphologyEx(black_noise_img, cv2.MORPH_CLOSE, kernel)
displayImage(closing)

Morphological gradient (takes the difference between dilation and erosion of the image):

<sup>(method of edge detection)</sup>

In [ ]:
img = loadImage()
gradient = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)
displayImage(gradient)

## <span style="color: yellowgreen;">7. </span>Gradients.

Image gradient is a directional change in the intensity or color in the image.

- In this lecture we will be mainly exploring basic Sobel-Feldman Operators (often called Sobel for short),
- Later on in the course we will expand on this operator for general edge detection,
- Gradients can be calculated in a specific direction.

The operator ises a 3x3 kernels which are convolved with the original image to calculate approximations of the derivatives- one for horizontal change and one for vertical.

<span style="font-size: 10px; text-decoration: none;">[more info](https://en.wikipedia.org/wiki/Sobel_operator)</span>
<!-- $\text{}_{(https://en.wikipedia.org/wiki/Sobel\_operator)}$ -->
$$
\nabla _x = 
\begin{bmatrix}
+1 & 0 & -1 \\[0.3em]
+2 & 0 & -2 \\[0.3em]
+1 & 0 & -1
\end{bmatrix}
\times A \text{}\quad\text{and}\quad \nabla _y = 
\begin{bmatrix}
+1 & +2 & +1 \\[0.3em]
0 & 0 & 0 \\[0.3em]
-1 & -2 & -1
\end{bmatrix}
\times A
$$
For our usecase we will focus on understanding the syntax of using Sobel with OpenCV.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def displayImage(img):
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap='gray')

In [ ]:
img = cv2.imread('./Computer-Vision-with-Python/DATA/sudoku.jpg', 0) # 0 => read in grayscale
displayImage(img)

Using Sobel operators in OpenCV:

In [ ]:
# depth is the precision of each pixel => CV_64F means 64 floating point precision
# dx, dy - derivatives
sobelx = cv2.Sobel(img, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
sobely = cv2.Sobel(img, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)
# displayImage(sobelx)
displayImage(sobely)

Gradient using Laplacian derivatives:

In [ ]:
laplacian = cv2.Laplacian(img, cv2.CV_64F)
displayImage(laplacian)

In [ ]:
blended = cv2.addWeighted(sobelx, 1, sobely, 1, 0)
# displayImage(blended)

ret, th1 = cv2.threshold(blended, 100, 255, cv2.THRESH_BINARY_INV)
displayImage(th1)
# th2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 8)
# displayImage(th2)

In [ ]:
kernel = np.ones((4,4),np.uint8)
gradient = cv2.morphologyEx(blended, cv2.MORPH_GRADIENT, kernel)
displayImage(gradient)

## <span style="color: yellowgreen;">8. </span>Histograms.

### <span style="color: royalblue;">a) </span>Histograms part one

- Histogram is a visual representation of the distribution of a continuous feature.
- For images we can display the frquency of values of colors.
    - Each channel has values between 0 and 255
    - We can plot these as 3 histograms on top of each other to see how much of each channel there is.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

If we perform calculations on an image, we have to stick to the cv2's expected BGR colorspace. Because of that we should create separate var image to show it in RGB after conversion.

In [ ]:
dark_horse = cv2.imread('./Computer-Vision-with-Python/DATA/horse.jpg') # original BGR for OpenCV
show_horse = cv2.cvtColor(dark_horse, cv2.COLOR_BGR2RGB) # converted to RGB for show

rainbow = cv2.imread('./Computer-Vision-with-Python/DATA/rainbow.jpg')
show_rainbow =cv2.cvtColor(rainbow, cv2.COLOR_BGR2RGB)

blue_bricks = cv2.imread('./Computer-Vision-with-Python/DATA/bricks.jpg')
show_bricks = cv2.cvtColor(blue_bricks, cv2.COLOR_BGR2RGB)

In [ ]:
# passing the image we want to calculate for as a list
# channel 0 of BGR => B
# optional mask parameter => we want to apply the mask only to part of the image
# histSize => upper limit
hist_values = cv2.calcHist([blue_bricks], channels=[0], mask=None, histSize=[256], ranges=[0, 256])
hist_values = cv2.calcHist([dark_horse], channels=[0], mask=None, histSize=[256], ranges=[0, 256])
plt.plot(hist_values)

In [ ]:
img = dark_horse
color = ('b', 'g', 'r')
for i, col in enumerate(color):
    histr = cv2.calcHist([img], [i], None, [256], [0, 256])
    plt.plot(histr, color=col)
    plt.xlim([0, 256])
    plt.ylim([0, 50000])
plt.title('Histogram of the image')

### <span style="color: royalblue;">b) </span>Histograms part two - histogram of a masked section

Additional topics:
- Histograms on a masked portion of the image,
    - We can select an ROI and only calculate the color histogram of the masked section.
- Histogram equalization.
    - It is a method of contrast adjustment based on image's histogram,
    - Applying it reduces the color depth (shades, higher contrast).

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


rainbow = cv2.imread('./Computer-Vision-with-Python/DATA/rainbow.jpg')
show_rainbow =cv2.cvtColor(rainbow, cv2.COLOR_BGR2RGB)

In [ ]:
img = rainbow
# :2 => only first two indices
mask = np.zeros(img.shape[:2], np.uint8)
mask[300:400, 100:400] = 255

masked_img = cv2.bitwise_and(img, img, mask=mask)
show_masked_img = cv2.bitwise_and(show_rainbow, show_rainbow, mask=mask)
plt.imshow(show_masked_img)

In [ ]:
# src; channel; mask; histSize; ranges
hist_mask_values_red = cv2.calcHist([rainbow], [2], mask, [256], [0, 256])
plt.plot(hist_mask_values_red, 'r')
plt.title('WITH mask')

hist_values_red = cv2.calcHist([rainbow], [2], None, [256], [0, 256])
plt.plot(hist_values_red, 'r')
plt.title('WITHOUT mask')

### <span style="color: royalblue;">c) </span>Histograms part three - histogram equalization

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
gorilla = cv2.imread('./Computer-Vision-with-Python/DATA/gorilla.jpg', 0)

def display(img,cmap=None):
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap)

In [ ]:
display(gorilla,'gray')

In [ ]:
hist_values = cv2.calcHist([gorilla], [0], None, [256], [0, 256])
plt.plot(hist_values)
plt.title('Histogram GRAY')

Equalization:

In [ ]:
eq_gorilla = cv2.equalizeHist(gorilla)
# display(eq_girilla, 'gray')

hist_eq_values = cv2.calcHist([eq_gorilla], [0], None, [256], [0, 256])
plt.plot(hist_eq_values)
plt.title('Histogram EQUALIZED GRAY')

Equalize the color image:

In [ ]:
color_grilla = cv2.imread('./Computer-Vision-with-Python/DATA/gorilla.jpg')
show_color_grilla = cv2.cvtColor(color_grilla, cv2.COLOR_BGR2RGB)
color = ('b', 'g', 'r')


for i, col in enumerate(color):
    hist_color_values = cv2.calcHist([color_grilla], [i], None, [256], [0, 256])
    plt.plot(hist_color_values, col)
plt.title('Histogram COLOR')

In [ ]:
hsv_gorilla = cv2.cvtColor(color_grilla, cv2.COLOR_BGR2HSV)
hsv_gorilla[:,:,2].min() # value channel min == 0
# equalization:
hsv_gorilla[:,:,2] = cv2.equalizeHist(hsv_gorilla[:,:,2])
# histogram 4fun
color = ('b', 'g', 'r')
for i, col in enumerate(color):
    hist_hsv_eq_values = cv2.calcHist([hsv_gorilla], [i], None, [256], [0, 256])
    plt.plot(hist_hsv_eq_values, col)
plt.title('Histogram EQUALIZED HSV')


hsv_eq_gorilla = cv2.cvtColor(hsv_gorilla, cv2.COLOR_HSV2RGB)
display(hsv_eq_gorilla) # showing the higher contrast of the colored image

## <span style="color: yellowgreen;">9. </span>Image processing assessment.

In separate notebook:

D:\1KURSY\Kurs Python\OpenCV_DL\Notebooks\Assessments\3. 07-Image-Processing-Assessment.ipynb